In [ ]:
import clr
clr.AddReference("qc-synchronizer")
from QuantConnect.DataFleet.Sync import ProducerSynchronizer

In [ ]:
import os
import pandas as pd
import requests
import sys
from pathlib import Path
import tarfile
from time import sleep

%config Completer.use_jedi = False

API_KEY = os.environ['FRED_API_KEY']
BASE_URL = 'https://api.stlouisfed.org/fred/series/observations?file_type=json&observation_start=1998-01-01'

In [ ]:
def download_fred_symbol(symbol, api_key, session):
    """
    Download series from FRED for a given FRED symbol
    """
    url = BASE_URL + f'&api_key={api_key}&series_id={symbol}'
    ex = None
    for retries in range(5):
        try:
            return session.get(url)
        except Exception as e:
            ex = e
            print(f'Failed to download data from URL: {url} - retrying')
            sleep(1)
            
    raise Exception(f'Failed to download data from URL: {url} - max retries exceeded (5/5) -- {ex}')

In [ ]:
def get_dataframe_from_request(request):
    """
    From FRED request's response, get the data as pandas DataFrame.
    :return: pandas DataFrame with the values correctly parsed.
    """
    response = request.json()
    if 'observations' not in response:
        return None
        
    full_response = pd.DataFrame(response['observations'])
    data = full_response.loc[:, ['date', 'value']]
    data['value'] = pd.to_numeric(full_response.value, errors='coerce')
    data = data.set_index(pd.to_datetime(data.date, errors='coerce'), drop=True)
    data.drop('date', axis=1, inplace=True)
    return data.dropna()

In [ ]:
destination_folder = Path('/temp-output-directory/alternative/fred')
destination_folder.mkdir(parents=True, exist_ok=True)

symbol_list_file = Path('./fred_implemented_series.csv')
symbol_list = pd.read_csv(symbol_list_file.resolve(), sep='|', names=['Id', 'Name', 'Unit'])
session = requests.Session()

for row in symbol_list.iterrows():
    symbol = row[1].Id
    name = row[1].Name

    print(f'Downloading {name} ... ')

    request = download_fred_symbol(symbol, API_KEY, session)
    df = get_dataframe_from_request(request)
    if df is None:
        print(f'Download failed for {name} - observations were null')
        continue

    destination_file = destination_folder / f'{symbol.lower()}.csv'
    df.to_csv(destination_file.resolve(), header=None, date_format='%Y%m%d')

    sleep(1)

In [ ]:
!aws s3 sync /temp-output-directory/ s3://cache.quantconnect.com/